In [9]:
import gymnasium as gym
import torch
import numpy as np
import dedalus.public as d3
import logging
import h5py
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from rbc_env import DedalusRBC_Env

In [10]:
#env = make_vec_env(DedalusRBC_Env, n_envs=30, seed=0, vec_env_cls=SubprocVecEnv)
gym.envs.register(id='rbc', entry_point=DedalusRBC_Env)
env = gym.make('rbc', render_mode="human")

/home/alexskillen/RB_DRL/.venv/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment rbc already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [11]:
model = PPO.load("logs/rl_model_60000_steps", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=3)

In [13]:
vec_env = model.get_env()
obs = vec_env.reset()
for i in range(248):
    action, _ = model.predict(obs, deterministic=False)
    obs, _, _, _ = vec_env.step(action)
    T = env.unwrapped.problem.variables[1]['g']
    fig, ax = plt.subplots()
    c = ax.imshow(np.transpose(T), aspect=1/np.pi, origin="lower", vmin=0., vmax=1.4)
    fig.colorbar(c)
    plt.title('$Nu=$'+str(np.round(env.unwrapped.fp.properties['Nu']['g'].flatten()[-1], 2)))
    plt.savefig('figs/'+str(i)+'.png')
    plt.close()

In [33]:
env.unwrapped.solver.stop_sim_time *= 2
for i in range(248):
    action, _ = model.predict(obs)
    obs, _, _, _ = vec_env.step(action*0)
    T = env.unwrapped.problem.variables[1]['g']
    fig, ax = plt.subplots()
    c = ax.imshow(np.transpose(T), aspect=1/np.pi, origin="lower", vmin=0., vmax=1.4)
    fig.colorbar(c)
    plt.title('$Nu=$'+str(np.round(env.unwrapped.fp.properties['Nu']['g'].flatten()[-1], 2)))
    plt.savefig('figs/'+str(248+i)+'.png')
    plt.close()

In [12]:
model.predict(obs)

(array([[ 0.581022  ,  1.        , -0.11605576, -0.89824307, -0.51246285,
          0.01351704, -0.29225165, -0.47820076, -1.        , -0.35339993]],
       dtype=float32),
 None)

In [16]:
np.average(env.unwrapped.g['g'].flatten())

0.924336564540863

In [60]:
def setBC(self, action):
    #lower BC
    self.act = action
    Tp = action - np.mean(action)

    Tp /= max(1., np.abs(Tp).max()/0.75)

    Tp = np.repeat(Tp, 3)
    self.Tp = Tp
    #copy last action due to periodicity
    Tp = np.append(Tp, Tp[0])
    xp=np.linspace(0, self.sim_metadata['Lx'], len(Tp))
    self.Tp4 = Tp
    T = np.interp(self.x, xp, Tp)

    self.g['g'] = T
    self.g['g'] += 1.0

In [18]:
r=DedalusRBC_Env()
r.reset()

(array([0.6372836 , 0.67803115, 0.72901434, 0.78587717, 0.84508824,
        0.895761  , 0.90977174, 0.8751388 , 0.817765  , 0.75878936,
        0.70408314, 0.65743613, 0.62384784, 0.60831445, 0.6136576 ,
        0.6388865 , 0.679814  , 0.73122096, 0.78868574, 0.84836066,
        0.89793885, 0.90848213, 0.87113047, 0.81327957, 0.75461996,
        0.7003652 , 0.6542511 , 0.62123996, 0.60624003, 0.6119504 ,
        0.4050605 , 0.45169804, 0.5147871 , 0.59739274, 0.6991035 ,
        0.7979048 , 0.82710856, 0.75673187, 0.6506971 , 0.55683434,
        0.4836747 , 0.42884716, 0.3903771 , 0.37186638, 0.37826103,
        0.4075861 , 0.45444798, 0.518331  , 0.6023878 , 0.705592  ,
        0.80262995, 0.82440877, 0.74865294, 0.64253974, 0.5500985 ,
        0.4781895 , 0.4242234 , 0.38642228, 0.3685791 , 0.3755214 ,
        0.36976212, 0.41169652, 0.45518366, 0.5187968 , 0.6184125 ,
        0.7310128 , 0.76626414, 0.6833503 , 0.5692878 , 0.4868052 ,
        0.4351724 , 0.39355686, 0.35127196, 0.32

In [71]:
setBC(r,r.action_space.sample())
r.g['g']

array([[1.59974205],
       [1.59974205],
       [1.59974205],
       [1.59974205],
       [1.59974205],
       [1.59974205],
       [1.59974205],
       [1.5797252 ],
       [1.51967465],
       [1.45962409],
       [1.39957353],
       [1.39957353],
       [1.39957353],
       [1.39957353],
       [1.39957353],
       [1.39957353],
       [1.39957353],
       [1.30601215],
       [1.025328  ],
       [0.74464385],
       [0.46395969],
       [0.46395969],
       [0.46395969],
       [0.46395969],
       [0.46395969],
       [0.46395969],
       [0.46395969],
       [0.58790336],
       [0.95973434],
       [1.33156533],
       [1.70339632],
       [1.70339632],
       [1.70339632],
       [1.70339632],
       [1.70339632],
       [1.70339632],
       [1.70339632],
       [1.70805669],
       [1.72203779],
       [1.7360189 ],
       [1.75      ],
       [1.75      ],
       [1.75      ],
       [1.75      ],
       [1.75      ],
       [1.75      ],
       [1.75      ],
       [1.644

In [70]:
np.average(r.g['g'].flatten())

1.000000011920929

In [58]:
r.Tp2

array([-0.5427242 ,  0.4604698 ,  0.35989147, -0.44864205, -0.58209145,
        0.75      ,  0.12681453,  0.75      , -0.75      , -0.48647383],
      dtype=float32)

In [59]:
r.act - np.mean(r.act)

array([-0.5427242 ,  0.4604698 ,  0.35989147, -0.44864205, -0.58209145,
        1.0501978 ,  0.12681453,  0.9067396 , -0.84418166, -0.48647383],
      dtype=float32)